# InfCtx trainer validation
This model being trained has the same settings as raven 1B5 model.
- Layer count: 24
- Embed size: 2048

The goal is to validate loss rate change, across the exact same hyper parameters with the following
- 1024 data chunk size
- same learningrate / weightdecay / seed
- "teven/enwiki_10k" dataset, chunked to 1024 token sizes

With only the change in training context size
- 1024 context vs 128 context

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Preparing the init model and test dataset

In [ ]:
# First lets setup the various directories, and get the blank init model, these init model was generated
# using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
# As such I have preinitialized these blank models and uploaded them to HF for convinence
!mkdir -p ../../model/
!mkdir -p ../../datapath/
!mkdir -p ../../checkpoint/
!rm -rf ../../model/Echo-A-1B5-Init.pth
!cd ../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
!ls -alh ../../model/Echo-A-1B5-Init.pth

In [9]:
# Lets preload the requried dataset
!cd ../../RWKV-v4neo && python3 preload_dataset.py ../notebook/trainer-validation/infctx-validation-dryrun.yaml

Found cached dataset parquet (/home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 986.66it/s]
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-3d43d1724bef83d7_*_of_00016.arrow
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-5033407f38c97f24.arrow
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-78e7f3a5f1679aa4_*_of_00016.arrow